## Problem

1. One class test bonus will be given if anyone solve input output variations (you need to be imaginative plus clever here) of CMOS nand gate for different parameters.

2. You can take this code as the starting point. But you need to develop a strategy or algorithm to solve the problem.

3. The time limit is upto last week of this semester.

4. You can write it in any language preferably python3 in jupyter lab.

5. You have to demonstrate the result to me in person and give necessary explanation about your program.

6. If you failed to do so you wouldn't get any bonus means here you will get no partial credit for your failed attempt. So think carefully.

7. If anyone doesn't want to participate in this, you are welcomed to ignore this challenge. There is no penalty for not attempting.

In [554]:
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from math import pi
from mpl_toolkits import mplot3d

# CMOS NAND
![title](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSHF8_w-GAwLl-ETJ-rVC0-uUqTDPDXXG61bNoQdNqbXR3ZdREO)

# Cadence Simulation
![title](CKT_NAND_Cadence.png)
![title](VTC_NAND_Cadence.png)

In [555]:
vd = np.linspace(0,5,100)    # Drain voltage

vt_n  = 1                    # threshold voltage for NMOS
vt_p  = 1                    # threshold voltage for PMOS

beta_n = 30e-6               # beta for NMOS
beta_p = 30e-6               # beta for PMOS

ids_n1 = np.zeros(100)       # for A nmos
ids_p1 = np.zeros(100)       # for A pmos

ids_n2 = np.zeros(100)       # for B nmos
ids_p2 = np.zeros(100)       # for B pmos

ids_n = np.zeros(100)        # total current through pull-down circuit
ids_p = np.zeros(100)        # total current through pull-up circuit

lambda_n = 0.3               # Channel modulation effect
lambda_p = 0.1               # Channel modulation effect

In [556]:
def current(vgs, vds, beta_n, beta_p):
    
    ids_n_temp = np.zeros(100)
    ids_p_temp = np.zeros(100)
    
    for i, v in enumerate(vds):
        
        # For NMOS
        if vgs <= vt_n:   #  cutoff 
            ids_n_temp[i] = 0
        elif v <= (vgs - vt_n):    # linear region
            ids_n_temp[i] = (beta_n * (vgs - vt_n - v / 2) * v)
        else:   # saturation region
            ids_n_temp[i] = (beta_n / 2 * (vgs - vt_n) ** 2) * (1 + v * lambda_n)
            
            
            
        # For PMOS
        vsd = 5 - v
        vsg = 5 - vgs
        if vsg <= vt_p:  # cutoff
            ids_p_temp[i] = 0
        elif vsd <= (vsg - vt_p): # linear region
            ids_p_temp[i] = (beta_p * (vsg - vt_p - vsd / 2) * vsd)
        else:   # saturation region
            ids_p_temp[i] = (beta_p / 2 * (vsg - vt_p) ** 2) * (1 + vsd * lambda_p)
            
    return ids_n_temp, ids_p_temp

In [557]:
def finding_intersection(vin_a, vin_b, vds, beta_n, beta_p):
    ids_n1, ids_p1 = current(vin_a, vds, beta_n, beta_p)
    ids_n2, ids_p2 = current(vin_b, vds, beta_n, beta_p)
    
    for i in range(len(ids_n)):
        ids_n[i] = min(ids_n1[i], ids_n2[i])
        ids_p[i] = ids_p1[i] + ids_p2[i]
    
    idiff = ids_n - ids_p
    vv = 0
    idd = 0
    for i in range(1, len(ids_p) - 1):
        if idiff[i] * idiff[i + 1] <= 0:
            vv = vds[i]
            idd = ids_n[i]
#             print(vv,idd)
            break
    return vv, idd, ids_n, ids_p

In [558]:
vin_a = np.linspace(0,5,100)
vin_b = 5
v_out = np.zeros(100)
i_out = np.zeros(100)
for i in range(len(vin_a)):
        vv, idd, ids_n, ids_p = finding_intersection(vin_a[i], vin_b, vd, beta_n, beta_p)
        v_out[i] = vv
        i_out[i] = idd

In [559]:
def drawing_plot(input_a, input_b):
    
    # VTC
    for i in range(len(vin_a)):
        vv, idd, ids_n, ids_p = finding_intersection(vin_a[i], input_b, vd, beta_n, beta_p)
        v_out[i] = vv
        i_out[i] = idd
    fig=plt.figure(figsize=(9, 6), dpi= 80)
    plt.subplot(222)
    plt.axis([0, 5, 0, 5])
    plt.title('Transfer characteristics')
    plt.plot(vin_a,v_out)
    plt.xlabel('Voltage at A (V)')
    plt.ylabel('Output Voltage (V)')
    vv, idd, ids_n, ids_p = finding_intersection(input_a, input_b, vd, beta_n, beta_p)
    plt.plot(input_a,vv,'ro')
    
    # IVC
    vv, idd, ids_n, ids_p = finding_intersection(input_a, input_b, vd, beta_n, beta_p)
    plt.subplot(221)
    plt.title('I-V characteristics')
    plt.plot(vd,ids_n*1000)
    plt.xlabel('Drain Voltage (V)')
    plt.ylabel('Drain Current (mA)')
    plt.plot(vd,ids_p*1000)
    plt.legend(['NMOS', 'PMOS'])
    plt.plot(vv,idd*1000,'ro')
    plt.axis([0,5,0,0.2])
    
    #
    plt.subplot(223)
    plt.title('Current')
    plt.plot(vin_a,i_out*1000)
    plt.xlabel('Voltage at A (V)')
    plt.ylabel('Drain Current (mA)')
    plt.plot(input_a,idd*1000,'ro')
    
    plt.subplot(224)
    plt.title('Power consumption')
    plt.plot(vin_a,5*i_out*1000)
    plt.xlabel('Voltage at A (V)')
    plt.ylabel('Power (mW)')
    plt.plot(input_a,5*idd*1000,'ro')
    plt.tight_layout()
    plt.show()
#     plt.plot(vgs,vv,'ro')

In [560]:
interact(drawing_plot, input_a =(0,5,0.1), input_b =(0,5,0.1))

interactive(children=(FloatSlider(value=2.0, description='input_a', max=5.0), FloatSlider(value=2.0, descripti…

<function __main__.drawing_plot(input_a, input_b)>